In [143]:
import pandas as pd

In [144]:
from utils import get_dir_path
from pathlib import Path
dir_path = get_dir_path()

In [145]:
wacc = pd.read_csv(Path(dir_path,'csv/My Wacc Report.csv'))
shares = pd.read_csv(Path(dir_path,'csv/My Shares Values.csv'))

In [146]:
merged_df = pd.merge(wacc, shares, left_on='Scrip Name', right_on='Scrip')
result = merged_df[['Scrip Name', 'Current Balance', 'WACC Rate']].copy()
result['WACC Rate'] = result['WACC Rate'].round(2)
result['Current Balance'] = result['Current Balance'].astype(float)


In [147]:
import nest_asyncio
nest_asyncio.apply()
from database.models import  Scripts
from database.session import  get_db
from sqlalchemy.orm import selectinload
from sqlalchemy import select
async with get_db() as db:
    scripts = (await db.execute(select(Scripts).filter(Scripts.ticker.in_(result['Scrip Name'])).options(selectinload(Scripts.script_details)))).scalars().all()
# result['LTP'] = result['Scrip Name'].apply(lambda name: next((scripts.filter(Scripts.ticker==name).first().script_details.last_traded_price), None)) 

In [148]:
result['LTP'] = result['Scrip Name'].apply(lambda name: next((script.script_details.last_traded_price for script in scripts if script.ticker == name), None)) 


In [149]:
result['Investment'] = result['Current Balance'] * result['WACC Rate']
result['Investment'] = result['Investment'].round(2)

result['Current Value'] = result['Current Balance'] * result['LTP']
result['Current Value'] = result['Current Value'].round(2)

result['Profit/Loss'] = result['Current Value'] - result['Investment']
result['Profit/Loss'] = result['Profit/Loss'].round(2)

result['Profit/Loss %'] = (result['Profit/Loss'] / result['Investment']) * 100
result['Profit/Loss %'] = result['Profit/Loss %'].round(2)


result = result.sort_values(by=['Investment', 'Profit/Loss %'], ascending=[False, False])

totals = result[['Investment', 'Current Value', 'Profit/Loss']].sum()
totals['Scrip Name'] = 'Total'
totals['Current Balance'] = ''
totals['WACC Rate'] = ''
totals['LTP'] = ''
totals['Profit/Loss %'] = (totals['Profit/Loss'] / totals['Investment']) * 100
totals['Profit/Loss %'] = round(totals['Profit/Loss %'],2)

result = pd.concat([result, totals.to_frame().T], ignore_index=True)

In [150]:
result.to_csv(Path(dir_path,'csv/My Wacc Rate.csv'), index=False)
result.to_excel(Path(dir_path,'csv/My Wacc Rate.xlsx'), index=False)

In [151]:
import openpyxl
from openpyxl.styles import PatternFill

wb = openpyxl.load_workbook(Path(dir_path,'csv/My Wacc Rate.xlsx'))
ws = wb.active

# Set the width of columns
column_widths = {
    'A': 15,
    'B': 20,
    'C': 15,
    'D': 10,
    'E': 15,
    'F': 15,
    'G': 15,
    'H': 15
}

for col, width in column_widths.items():
    ws.column_dimensions[col].width = width

red_fill_1 = PatternFill(start_color="FFCCCC", end_color="FFCCCC", fill_type="solid")
red_fill_2 = PatternFill(start_color="FF6666", end_color="FF6666", fill_type="solid")
red_fill_3 = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")

green_fill_1 = PatternFill(start_color="CCFFCC", end_color="CCFFCC", fill_type="solid")
green_fill_2 = PatternFill(start_color="66FF66", end_color="66FF66", fill_type="solid")
green_fill_3 = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")

neutral_fill = PatternFill(start_color="FFFFCC", end_color="FFFFCC", fill_type="solid")

profit_loss_col = None
current_value_col = None
for idx, cell in enumerate(ws[1]):
    if cell.value == 'Profit/Loss %':
        profit_loss_col = idx + 1
    if cell.value == 'Current Balance':
        current_value_col = idx + 1

for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    profit_loss_value = row[profit_loss_col - 1].value
    current_value_value = row[current_value_col - 1].value
    if current_value_value and current_value_value == 10:
        fill = neutral_fill
        for cell in row:
            cell.fill = fill
        continue
    if profit_loss_value:
        if profit_loss_value > 50:
            fill = green_fill_3
        elif profit_loss_value > 20:
            fill = green_fill_2
        elif profit_loss_value > 0:
            fill = green_fill_1
        elif profit_loss_value < -50:
            fill = red_fill_3
        elif profit_loss_value < -20:
            fill = red_fill_2
        elif profit_loss_value < 0:
            fill = red_fill_1
        else:
            fill = neutral_fill
        for cell in row:
            cell.fill = fill

wb.save(Path(dir_path,'csv/My Wacc Rate.xlsx'))

In [ ]:
from database.models import  S